In [1]:
import pandas as pd
import random
from caboose_nbr.pernir import Pernir
from caboose_nbr.evaluate_recommendation import evaluate

In [2]:
train_baskets = pd.read_csv('data/instacart_30k/train_baskets.csv.gz')
test_samples = pd.read_csv('data/instacart_30k/test_samples.csv')

In [3]:
all_users = train_baskets['user_id'].tolist()
sample_users = random.sample(all_users,1000)

In [4]:
train_baskets = train_baskets[train_baskets['user_id'].isin(sample_users)]
test_samples = test_samples[test_samples['user_id'].isin(sample_users)]
test_samples = test_samples.groupby('user_id').sample(1)

In [5]:
print(train_baskets.shape)
print(test_samples.shape)
print(train_baskets[['user_id','item_id']].drop_duplicates().shape)

(404214, 10)
(961, 5)
(119222, 2)


In [6]:
pernir_similaripy = Pernir(train_baskets, test_samples,'similaripy')
pernir_similaripy.train()

start of knn


Done: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 961/961 [00:00<00:00, 19404.24it/s]


knn finished


In [7]:
pernir_similaripy_preds , sim_scores = pernir_similaripy.predict()

num test users: 961
0 samples passed
500 samples passed


In [8]:
pernir_caboose = Pernir(train_baskets, test_samples,'caboose')
pernir_caboose.train()

start of knn
knn finished


--Creating transpose of R...
--Computing row norms...
--Configuring for top-k -- num_threads: 4; pinning? false;
--Scheduling parallel top-k computation...


In [9]:
pernir_caboose_preds , caboose_scores = pernir_caboose.predict()

num test users: 961
0 samples passed
500 samples passed


In [10]:
def sanity_check(users,default_model,caboose_model,default_preds, caboose_preds):
    bad_users  = []
    for user in users:
        if set(default_model.user_neighbors[user]) != set(caboose_model.user_neighbors[user]):
            bad_users.append(user)
    print('bad users:',len(bad_users))
    bad_preds  = []
    for i in range(len(default_preds)):
        if default_preds[i] != caboose_preds[i]:
            bad_preds.append(i)
    print('bad predictions:',len(bad_preds))

In [11]:
sanity_check(test_samples['user_id'].tolist(),
             pernir_similaripy,pernir_caboose,
             pernir_similaripy_preds,pernir_caboose_preds)

bad users: 0
bad predictions: 0


In [12]:
interactions_to_forget = train_baskets[['user_id','item_id']].drop_duplicates().sample(5000)

In [13]:
pernir_similaripy.forget_interactions(interactions_to_forget.values.tolist())
pernir_similaripy_preds_after_forget , _ = pernir_similaripy.predict()

/Users/mozhdeh/Documents/caboose/caboose_nbr/pernir.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.train_baskets.drop('user_item', axis=1, inplace=True)


start of knn


Done: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 961/961 [00:00<00:00, 22475.33it/s]


knn finished
num test users: 961
0 samples passed
500 samples passed


In [14]:
pernir_caboose.forget_interactions(interactions_to_forget.values.tolist())
pernir_caboose_preds_after_forget , _ = pernir_caboose.predict()

/Users/mozhdeh/Documents/caboose/caboose_nbr/pernir.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.train_baskets.drop('user_item', axis=1, inplace=True)


num test users: 961
0 samples passed
500 samples passed


In [15]:
sanity_check(test_samples['user_id'].tolist(),
             pernir_similaripy,pernir_caboose,
             pernir_similaripy_preds_after_forget[0],pernir_caboose_preds_after_forget[0])

bad users: 0
bad predictions: 0
